# Long Sentence Resolution - Demonstrate Evaluation Metrics

# Documentation

## Shortening

### ROUGE

#### Description

ROUGE, or Recall-Oriented Understudy for Gisting Evaluation, is a set of metrics and a software package used for evaluating automatic summarization and machine translation software in natural language processing. The metrics compare an automatically produced summary or translation against a reference or a set of references (human-produced) summary or translation. Note that ROUGE is case insensitive, meaning that upper case letters are treated the same way as lower case letters.

For more information please visit:

* https://en.wikipedia.org/wiki/ROUGE_(metric)
* https://huggingface.co/spaces/evaluate-metric/rouge

#### Citations

```
@inproceedings{lin-2004-rouge,
    title = "{ROUGE}: A Package for Automatic Evaluation of Summaries",
    author = "Lin, Chin-Yew",
    booktitle = "Text Summarization Branches Out",
    month = jul,
    year = "2004",
    address = "Barcelona, Spain",
    publisher = "Association for Computational Linguistics",
    url = "https://www.aclweb.org/anthology/W04-1013",
    pages = "74--81",
}
```

### Word-based Compression Rate


#### Description

Compression Rate (CR) is the ratio of the number of words in the compressed sentence to the number of words in the original sentence. As an example, a model with 0.38 CR value, means 62% of the words in the sentence were deleted. We aim for moderate CR values, as too low CR scores indicate overcompression, and too high CR values indicate under-compression.

\begin{align}
Compression\;Rate = \frac{\#words\;in\;compressed\;sentence}{\#words\;in\;original\;sentence}
\end{align}

### Word-based Accuracy

#### Description

Accuracy for shortening is the ratio of the number of resolved (successfully shortened) sentences to all the sentences. A resolved sentence is any sentence where its number of words is less than or equal to a defined threshold (e.g. 16).

\begin{align}
Accuracy = \frac{\#resolved\;sentences}{\#all\;sentences}
\end{align}

## Splitting

### BLEU

#### Description

BLEU (bilingual evaluation understudy) is an algorithm for evaluating the quality of text which has been machine-translated from one natural language to another. Quality is considered to be the correspondence between a machine's output and that of a human: "the closer a machine translation is to a professional human translation, the better it is" – this is the central idea behind BLEU. BLEU was one of the first metrics to claim a high correlation with human judgements of quality, and remains one of the most popular automated and inexpensive metrics.

Scores are calculated for individual translated segments—generally sentences—by comparing them with a set of good quality reference translations. Those scores are then averaged over the whole corpus to reach an estimate of the translation's overall quality. Intelligibility or grammatical correctness are not taken into account.

For more information please visit:

* https://en.wikipedia.org/wiki/BLEU
* https://huggingface.co/spaces/evaluate-metric/bleu

#### Citations

```
@INPROCEEDINGS{Papineni02bleu:a,
    author = {Kishore Papineni and Salim Roukos and Todd Ward and Wei-jing Zhu},
    title = {BLEU: a Method for Automatic Evaluation of Machine Translation},
    booktitle = {},
    year = {2002},
    pages = {311--318}
}

@inproceedings{lin-och-2004-orange,
    title = "{ORANGE}: a Method for Evaluating Automatic Evaluation Metrics for Machine Translation",
    author = "Lin, Chin-Yew and Och, Franz Josef",
    booktitle = "{COLING} 2004: Proceedings of the 20th International Conference on Computational Linguistics",
    month = "aug 23{--}aug 27",
    year = "2004",
    address = "Geneva, Switzerland",
    publisher = "COLING",
    url = "https://www.aclweb.org/anthology/C04-1072",
    pages = "501--507",
}
```

### SacreBLEU

#### Description

SacreBLEU provides hassle-free computation of shareable, comparable, and reproducible **BLEU** scores. Inspired by Rico Sennrich’s `multi-bleu-detok.perl`, it produces the official Workshop on Machine Translation (WMT) scores but works with plain text. It also knows all the standard test sets and handles downloading, processing, and tokenization.

For more information please visit:

* https://github.com/mjpost/sacreBLEU
* https://huggingface.co/spaces/evaluate-metric/sacrebleu

#### Citations

```
@inproceedings{post-2018-call,
    title = "A Call for Clarity in Reporting {BLEU} Scores",
    author = "Post, Matt",
    booktitle = "Proceedings of the Third Conference on Machine Translation: Research Papers",
    month = oct,
    year = "2018",
    address = "Belgium, Brussels",
    publisher = "Association for Computational Linguistics",
    url = "https://www.aclweb.org/anthology/W18-6319",
    pages = "186--191",
}
```

### Word-based Accuracy

#### Description

Accuracy for splitting first calculates for each i<sup>th</sup> sample the ratio of the resolved (successfully shortened) sentences to all the n sentences in the sample. Accuracy is the ratio of the sum of all the aforementioned ratios to all the samples. A resolved sentence is any sentence where its number of words is less than or equal to a defined threshold (e.g. 16).

\begin{align}
Accuracy = \frac{\sum_{i=1}^n \frac{\#resolved\;sentences\;in\;i^{th}\;sample}{\#sentences\;in\;i^{th}\;sample}}{n}
\end{align}

## Both

### BERTScore

#### Description

BERTScore leverages the pre-trained contextual embeddings from BERT and matches words in candidate and reference sentences by cosine similarity. It has been shown to correlate with human judgment on sentence-level and system-level evaluation. Moreover, BERTScore computes precision, recall, and F1 measure, which can be useful for evaluating different language generation tasks.

For more information please visit:

* https://github.com/Tiiiger/bert_score#readme

#### Citations

```
@inproceedings{bert-score,
  title={BERTScore: Evaluating Text Generation with BERT},
  author={Tianyi Zhang* and Varsha Kishore* and Felix Wu* and Kilian Q. Weinberger and Yoav Artzi},
  booktitle={International Conference on Learning Representations},
  year={2020},
  url={https://openreview.net/forum?id=SkeHuCVFDr}
}
```

# Installations

In [ ]:
!pip install evaluate==0.3.0    \
             rouge-score==0.1.2 \
             bert-score==0.3.11 \
             sacrebleu==2.3.1   \
             spacy==3.4.1

# Imports

In [ ]:
import json, evaluate

from typing import List, Union, Dict

from spacy.lang.en import English

from spacy.tokens import Doc, Token, Span

import numpy as np

import pandas as pd

# Functions / Classes

In [ ]:
class DocumentAnalysis:
  def __init__ (self):
    self._linguistics_model = English()

    self._linguistics_model.add_pipe("sentencizer")

  def extract_n_words_from_documents(self, documents: Union[List[str], np.ndarray, pd.Series]) -> np.ndarray:
    return np.array(list(map(self._extract_n_words_from_document, self._linguistics_model.pipe(documents, batch_size=50))))

  def extract_sentence_n_words_from_documents(self, documents: Union[List[str], np.ndarray, pd.Series]) -> pd.Series:
    series_index = None if type(documents) != pd.Series else documents.index

    return pd.Series(list(map(self._extract_sentence_n_words_from_document, self._linguistics_model.pipe(documents, batch_size=50))), index=series_index)

  def _extract_sentence_n_words_from_document(self, doc: Doc) -> List[int]:
    return list(map(self._extract_n_words_from_span, doc.sents))

  def _extract_n_words_from_span(self, span: Span) -> int:
    return sum(map(self._token_is_word, span))

  def _extract_n_words_from_document(self, doc: Doc) -> int:
    return sum(map(self._token_is_word, doc))

  def _token_is_word(self, token: Token) -> bool:
    return not any([ token.is_bracket,
                     token.is_punct,
                     token.is_currency,
                     token.is_quote,
                     token.is_space ]) # https://spacy.io/api/token#attributes

class WordBasedCompressionRate:
  def __init__ (self):
    self._document_analysis = DocumentAnalysis()

  def compute(self, uncompressed_documents: Union[List[str], np.ndarray, pd.Series], compressed_documents: Union[List[str], np.ndarray, pd.Series], use_aggregator: bool = True) -> Union[Dict, float]:
    assert(len(uncompressed_documents) == len(compressed_documents))

    uncompressed_n_words = self._document_analysis.extract_n_words_from_documents(uncompressed_documents)

    compressed_n_words = self._document_analysis.extract_n_words_from_documents(compressed_documents)

    compression_rates = compressed_n_words / uncompressed_n_words

    return np.mean(compression_rates) if use_aggregator else {
        'compression_rates': compression_rates,
        'uncompressed_n_words': uncompressed_n_words,
        'compressed_n_words': compressed_n_words
    }

class SplittingWordBasedAccuracy:
  def __init__ (self, n_words_threshold: int):
    self._document_analysis = DocumentAnalysis()

    self._n_words_threshold = n_words_threshold

  def compute(self, documents: Union[List[str], np.ndarray, pd.Series]) -> float:
    if len(documents) == 0: return 0

    sample_level_accuracies = np.array(self._document_analysis.extract_sentence_n_words_from_documents(documents).apply(lambda o: sum(map(lambda l : l <= self._n_words_threshold, o)) / len(o)))

    return np.mean(sample_level_accuracies)

class ShorteningWordBasedAccuracy:
  def __init__ (self, n_words_threshold: int):
    self._document_analysis = DocumentAnalysis()

    self._n_words_threshold = n_words_threshold

  def compute(self, documents: Union[List[str], np.ndarray, pd.Series]) -> float:
    if len(documents) == 0: return 0

    return np.count_nonzero(self._document_analysis.extract_n_words_from_documents(documents) <= self._n_words_threshold) / len(documents)

# Demonstration

## Shortening

### Load Metrics

In [ ]:
rouge_metric = evaluate.load('rouge', module_type="metric")

shortening_word_based_accuracy_metric = ShorteningWordBasedAccuracy(n_words_threshold=16)

word_based_compression_rate_metric = WordBasedCompressionRate()

### ROUGE

In [ ]:
inputs = [
    "From the Vedantic point of view, these 11 yoga upanishads belong to the Vedic Shaka (school).",
    "He was born in Hagerstown, Maryland and died in Baltimore, Maryland at the age of 69."
]

golden_outputs = [
    "These 11 Yoga Upanishads belong to Vedantic shaka (school) from the Vedic point of view.",
    "Born in Hagerstown, Maryland, he died at the age of 69 in Baltimore, Maryland."
]

system_outputs_incorrect = [
    "He graduated from Phi Beta Kappa magna cum laude and was elected Harvard Law School.",
    "He graduated from Phi Beta Kappa magna cum laude and was elected Harvard Law School."
]

print(f"Compare inputs to golden outputs: {json.dumps(rouge_metric.compute(predictions=golden_outputs, references=inputs, rouge_types=['rouge1', 'rouge2', 'rougeL']), indent=True)}")

print(f"Compare inputs to incorrect system outputs: {json.dumps(rouge_metric.compute(predictions=system_outputs_incorrect, references=inputs, rouge_types=['rouge1', 'rouge2', 'rougeL']), indent=True)}")

### Word-based Accuracy


In [ ]:
system_outputs = [
    "Many cultures have stories describing the origin of the world and universe.",
    "These 11 Yoga Upanishads belong to Vedantic shaka (school) from the Vedic point of view.",
   
    "From the Vedantic point of view, these 11 yoga upanishads belong to the Vedic Shaka (school).",
    "Lady Gaga, who was recently on a UK TV talk show, has revealed that she is teaming up with Sir Elton John again for a new duet."
]

print(f'{np.round(100.0 * shortening_word_based_accuracy_metric.compute(system_outputs), 2)}% of the sentences have been resolved')

### Word-based Compression Rate


In [ ]:
uncompressed_inputs = [
  "Cambodian leader Hun Sen on Friday rejected opposition parties' demands for talks outside the country, accusing them of trying to 'internationalize' the political crisis.",
  "The radical group Islamic Jihad claimed responsibility Saturday for the suicide bombing of a crowded Jerusalem market and promised more attacks to try to block the new peace accord.",
  "Fewer than a dozen FBI agents will be sent to Iraq to secure and analyze evidence of the bombing."
]

compressed_outputs = [
  "Cambodian government rejects opposition's call for talks abroad.",
  "Islamic Jihad, not Hamas, perpetrator of market suicide bombing.",
  "Fewer than a dozen FBI agents were dispatched to secure and analyze evidence."
]

print(f'{np.round(100.0 * word_based_compression_rate_metric.compute(uncompressed_inputs, compressed_outputs), 2)}%')

## Splitting

### Load Metrics

In [ ]:
sacre_bleu_metric = evaluate.load('sacrebleu', module_type="metric")

splitting_word_based_accuracy_metric = SplittingWordBasedAccuracy(n_words_threshold=16)

### SacreBLEU

In [ ]:
inputs = [
  "A pedestrian tunnel allowed passengers to cross from northbound to southbound platforms and vice versa, but was closed (sealed off) in 2004 because of safety concerns.",
  "A person born in the presence of this condition is termed a manglik and the term manglik is a search criteria on some Indian and UAE dating websites."
]

golden_outputs = [
  "A pedestrian tunnel allows passengers to cross from northbound to southbound platforms and vice versa. This pedestrian tunnel was closed (sealed off) in 2004 because of safety concerns.",
  "A person born in the presence of this condition is termed a manglik. Manglik is a search criteria on some Indian and UAE dating websites."
]

system_outputs_partially_correct = [
  "A pedestrian tunnel allows passengers to cross from northbound to southbound platforms and vice versa. He performed with two childhood friends, Three of Clubs, and worked the Minnesota Renaissance Fair.",
  "A person born in the presence of this condition is termed a manglik. The Christmas hymns take approximately an additional ten minutes."
]

system_outputs_incorrect = [
  "A performer since age 14, Wee preferred to juggle with partners. He performed with two childhood friends, Three of Clubs, and worked the Minnesota Renaissance Fair.",
  "A performer since age 14, Wee preferred to juggle with partners. He performed with two childhood friends, Three of Clubs, and worked the Minnesota Renaissance Fair."
]

print(f"Compare inputs to golden outputs: {json.dumps(sacre_bleu_metric.compute(predictions=golden_outputs, references=inputs), indent=True)}")

print(f"Compare inputs to partially correct system outputs: {json.dumps(sacre_bleu_metric.compute(predictions=system_outputs_partially_correct, references=inputs), indent=True)}")

print(f"Compare inputs to incorrect system outputs: {json.dumps(sacre_bleu_metric.compute(predictions=system_outputs_incorrect, references=inputs), indent=True)}")

### Word-based Accuracy


In [ ]:
system_outputs = [
  "A pedestrian tunnel allows passengers to cross from northbound to southbound platforms and vice versa. This pedestrian tunnel was closed (sealed off) in 2004 because of safety concerns.",
  "A person born in the presence of this condition is termed a manglik. A pedestrian tunnel allowed passengers to cross from northbound to southbound platforms and vice versa, but was closed (sealed off) in 2004 because of safety concerns.",
  "A pedestrian tunnel allowed passengers to cross from northbound to southbound platforms and vice versa, but was closed (sealed off) in 2004 because of safety concerns.",
]

'''
  Sample 1: 2/2 sentences resolved = 1
  Sample 2: 1/2 sentences resolved = 0.5
  Sample 3: 0/1 sentences resolved = 0
  
  Accuracy = (1 + 0.5 + 0) / 3 = 0.5 = 50%
'''

print(f'{np.round(100.0 * splitting_word_based_accuracy_metric.compute(system_outputs), 2)}%')

## Both

### Load Metrics

In [ ]:
bertscore_metric = evaluate.load("bertscore", module_type="metric")

### BERTScore

In [ ]:
inputs = [
    "From the Vedantic point of view, these 11 yoga upanishads belong to the Vedic Shaka (school).",
    "During a recent appearance on TV, Beyonce disclosed that she will be working with Elton John."
]

golden_outputs = [
    "These 11 yoga upanishads are part of the Vedic Shaka (school) from the Vedantic perspective.",
    "Beyonce was recently on TV and has revealed she is teaming up with Elton John."
]

system_outputs_incorrect = [
    "He graduated from Phi Beta Kappa magna cum laude and was elected Harvard Law School.",
    "He graduated from Phi Beta Kappa magna cum laude and was elected Harvard Law School."
]

print(f"Compare inputs to golden outputs: {json.dumps(bertscore_metric.compute(predictions=golden_outputs, references=inputs, lang='en'), indent=True)}")

print(f"Compare inputs to incorrect system outputs: {json.dumps(bertscore_metric.compute(predictions=system_outputs_incorrect, references=inputs, lang='en'), indent=True)}")